In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 3037684217260,
 'week52high': 198.23,
 'week52low': 123.64,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.25392724715800075,
 'sharesOutstanding': 15728702000,
 'float': 0,
 'avg10Volume': 54667326,
 'avg30Volume': 52277009,
 'day200MovingAvg': 163.56,
 'day50MovingAvg': 187.27,
 'employees': 147000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.9264396314120724,
 'dividendYield': 0.0047969742215713375,
 'nextDividendDate': '',
 'exDividendDate': '2023-05-12',
 'nextEarningsDate': '2023-08-03',
 'peRatio': 32.20581012987564,
 'beta': 1.276297117301474,
 'maxChangePercent': 74.1917461553436,
 'year5ChangePercent': 3.2336451261456576,
 'year2ChangePercent': 0.340474944438969,
 'year1ChangePercent': 0.2728539332063093,
 'ytdChangePercent': 0.49280611684900655,
 'month6ChangePercent': 0.4068348001672497,
 'month3ChangePercent': 0.1621043156052202,
 'month1ChangePercent': 0.04388951948543318,
 'day30ChangePe

In [4]:
data['companyName']

'Apple Inc'

In [50]:
data['year1ChangePercent']

KeyError: 'year1ChangePercent'

In [9]:
data['year1ChangePercent']

0.2728539332063093

In [16]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
 import pandas as pd
import requests

# Assuming 'my_columns' is defined

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            try:
                latest_price = data[symbol]['quote']['latestPrice']
                year1_change_percent = data[symbol]['stats']['year1ChangePercent']
            except KeyError:
                # Skip the symbol if any of the data is missing
                continue

            final_dataframe = final_dataframe.append(
                pd.Series([symbol, latest_price, year1_change_percent, 'N/A'], index=my_columns),
                ignore_index=True
            )

print(final_dataframe)

    Ticker    Price One-Year Price Return Number of Shares to Buy
0        A   127.13              0.046172                     N/A
1      AAL    17.41              0.146614                     N/A
2      AAP    70.05             -0.623538                     N/A
3     AAPL  193.305              0.272854                     N/A
4     ABBV   143.64              0.001818                     N/A
..     ...      ...                   ...                     ...
496    YUM   136.87              0.140735                     N/A
497    ZBH    142.9              0.331786                     N/A
498   ZBRA   308.68             -0.064163                     N/A
499   ZION    36.18             -0.246262                     N/A
500    ZTS   183.28             -0.017709                     N/A

[501 rows x 4 columns]


In [52]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            latest_price = data[symbol]['quote'].get('latestPrice', None)
            year1_change_percent = data[symbol]['stats'].get('year1ChangePercent', None)

            if latest_price is not None and pd.notna(latest_price):
                final_dataframe = final_dataframe.append(
                    pd.Series([symbol, latest_price, year1_change_percent, 'N/A'], index=my_columns),
                    ignore_index=True
                )

# Fill missing values with the mean of the respective columns
final_dataframe['Price'].fillna(final_dataframe['Price'].mean(), inplace=True)
final_dataframe['One-Year Price Return'].fillna(final_dataframe['One-Year Price Return'].mean(), inplace=True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,126.980,0.046172,N/A
1,AAL,17.410,0.146614,N/A
2,AAP,70.055,-0.623538,N/A
3,AAPL,193.350,0.272854,N/A
4,ABBV,143.610,0.001818,N/A
...,...,...,...,...
496,YUM,136.870,0.140735,N/A
497,ZBH,142.900,0.331786,N/A
498,ZBRA,308.680,-0.064163,N/A
499,ZION,36.000,-0.246262,N/A


In [21]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.420,0.046172,N/A
1,AAL,17.220,0.146614,N/A
2,AAP,69.790,-0.623538,N/A
3,AAPL,193.000,0.272854,N/A
4,ABBV,143.610,0.001818,N/A
...,...,...,...,...
496,YUM,136.630,0.140735,N/A
497,ZBH,142.980,0.331786,N/A
498,ZBRA,307.885,-0.064163,N/A
499,ZION,36.990,-0.246262,N/A


In [22]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [23]:
import pandas as pd

# Assuming 'final_dataframe' is your DataFrame

# Step 1: Convert 'Price' and 'One-Year Price Return' columns to numeric
final_dataframe['Price'] = pd.to_numeric(final_dataframe['Price'], errors='coerce')
final_dataframe['One-Year Price Return'] = pd.to_numeric(final_dataframe['One-Year Price Return'], errors='coerce')

# Step 2: Fill missing, zero, and non-integer values with the mean
for column in ['Price', 'One-Year Price Return']:
    final_dataframe[column].fillna(final_dataframe[column].mean(), inplace=True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.420,0.046172,N/A
1,AAL,17.220,0.146614,N/A
2,AAP,69.790,-0.623538,N/A
3,AAPL,193.000,0.272854,N/A
4,ABBV,143.610,0.001818,N/A
...,...,...,...,...
496,YUM,136.630,0.140735,N/A
497,ZBH,142.980,0.331786,N/A
498,ZBRA,307.885,-0.064163,N/A
499,ZION,36.990,-0.246262,N/A


In [24]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [25]:
final_dataframe['One-Year Price Return'] = pd.to_numeric(final_dataframe['One-Year Price Return'], errors='coerce')

# Step 2: Sort the DataFrame
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)

# Step 3: Select the top 51 rows
final_dataframe = final_dataframe.head(51)

# Step 4: Reset the index
final_dataframe.reset_index(drop=True, inplace=True)

# Step 5: Display the resulting DataFrame
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.920,2.281287,N/A
1,FTI,17.300,1.759874,N/A
2,RCL,101.900,1.595226,N/A
3,NVDA,446.790,1.558676,N/A
4,GE,111.350,1.090050,N/A
5,NFLX,429.310,1.020976,N/A
6,AVGO,903.770,0.796676,N/A
7,PHM,78.260,0.768861,N/A
8,DHI,127.930,0.734793,N/A
9,CAH,94.250,0.733667,N/A


In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:5486584665
5486584665


In [27]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.920,2.281287,1346097
1,FTI,17.300,1.759874,6218502
2,RCL,101.900,1.595226,1055741
3,NVDA,446.790,1.558676,240784
4,GE,111.350,1.090050,966143
5,NFLX,429.310,1.020976,250588
6,AVGO,903.770,0.796676,119034
7,PHM,78.260,0.768861,1374649
8,DHI,127.930,0.734793,840929
9,CAH,94.250,0.733667,1141433


In [33]:
hqm_columns = [
    'Ticker', 'Price', 'Number of Shares to Buy',
    'One-Year Price Return', 'One-Year Return Percentile',
    'Six-Month Price Return', 'Six-Month Return Percentile',
    'Three-Month Price Return', 'Three-Month Return Percentile',
    'One-Month Price Return', 'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings[:5]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            try:
                latest_price = data[symbol]['quote']['latestPrice']
                year1_change_percent = data[symbol]['stats']['year1ChangePercent']
                month6_change_percent = data[symbol]['stats']['month6ChangePercent']
                month3_change_percent = data[symbol]['stats']['month3ChangePercent']
                month1_change_percent = data[symbol]['stats']['month1ChangePercent']
            except KeyError:
                # Skip the symbol if any of the data is missing
                continue

            hqm_dataframe = hqm_dataframe.append(
                pd.Series([symbol, latest_price, 'N/A', year1_change_percent, 'N/A',
                           month6_change_percent, 'N/A', month3_change_percent, 'N/A',
                           month1_change_percent, 'N/A', 'N/A'],
                          index=hqm_columns),
                ignore_index=True
            )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.42,N/A,0.046172,N/A,-0.198364,N/A,-0.07926,N/A,0.056092,N/A,N/A
1,AAL,17.33,N/A,0.146614,N/A,0.067319,N/A,0.309309,N/A,0.068627,N/A,N/A
2,AAP,69.63,N/A,-0.623538,N/A,-0.517678,N/A,-0.452689,N/A,0.033514,N/A,N/A
3,AAPL,193.11,N/A,0.272854,N/A,0.406835,N/A,0.162104,N/A,0.04389,N/A,N/A
4,ABBV,143.35,N/A,0.001818,N/A,-0.03009,N/A,-0.111127,N/A,0.043145,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,XLNX,194.92,N/A,0,N/A,0,N/A,0,N/A,0,N/A,N/A
492,XOM,103.74,N/A,0.198852,N/A,-0.072696,N/A,-0.098,N/A,0.006717,N/A,N/A
493,XRAY,40.88,N/A,0.145058,N/A,0.109847,N/A,0.000705,N/A,0.042293,N/A,N/A
494,XRX,15.49,N/A,0.049897,N/A,-0.038604,N/A,0.153423,N/A,0.110412,N/A,N/A


In [55]:
my_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
              'One-Year Return Percentile', 'Six-Month Price Return',
              'Six-Month Return Percentile', 'Three-Month Price Return',
              'Three-Month Return Percentile', 'One-Month Price Return',
              'One-Month Return Percentile', 'HQM Score']

# Create an empty DataFrame with the specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

In [56]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0     0.294118
1     0.529412
2     0.019608
3     0.745098
4     0.215686
5     0.921569
6     0.186275
7     0.333333
8     0.490196
9     0.784314
10    0.568627
11    0.509804
12    0.470588
13    0.372549
14     0.27451
15    0.137255
16    0.431373
17    0.901961
18    0.588235
19    0.607843
20    0.058824
21    0.823529
22    0.254902
23    0.235294
24    0.627451
25    0.705882
26    0.117647
27     0.72549
28    0.186275
29    0.843137
30    0.078431
31    0.647059
32    0.941176
33    0.156863
34    0.960784
35    0.039216
36    0.352941
37    0.980392
38    0.803922
39    0.764706
40    0.686275
41    0.882353
42     0.45098
43    0.862745
44    0.666667
45    0.392157
46    0.098039
47    0.411765
48     0.54902
49    0.313725
50         1.0
Name: One-Year Return Percentile, dtype: object
0     0.058824
1      0.54902
2     0.019608
3     0.921569
4     0.235294
5     0.784314
6     0.303922
7     0.333333
8     0.666667
9     0.941176
10     0.72549
11     0.27451
12    0

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.42,8507968853,0.046172,0.294118,-0.198364,0.058824,-0.07926,0.098039,0.056092,0.705882,N/A
1,AAL,17.33,62064479080,0.146614,0.529412,0.067319,0.54902,0.309309,0.960784,0.068627,0.764706,N/A
2,AAP,69.63,15447040391,-0.623538,0.019608,-0.517678,0.019608,-0.452689,0.019608,0.033514,0.45098,N/A
3,AAPL,193.11,5569765535,0.272854,0.745098,0.406835,0.921569,0.162104,0.823529,0.04389,0.568627,N/A
4,ABBV,143.35,7503156068,0.001818,0.215686,-0.03009,0.235294,-0.111127,0.039216,0.043145,0.529412,N/A
5,ABC,194.16,5539644738,0.387965,0.921569,0.19945,0.784314,0.168746,0.843137,0.04835,0.607843,N/A
6,ABMD,381.02,2822889671,0,0.186275,0,0.303922,0,0.264706,0,0.205882,N/A
7,ABT,113.77,9453963456,0.053077,0.333333,0.000956,0.333333,0.018272,0.313725,0.053105,0.647059,N/A
8,ACN,314.88,3415832769,0.127349,0.490196,0.122528,0.666667,0.138102,0.705882,-0.011959,0.137255,N/A
9,ADBE,523.5,2054589154,0.287404,0.784314,0.450362,0.941176,0.35928,0.980392,0.063846,0.72549,N/A


In [57]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.42,8507968853,0.046172,0.294118,-0.198364,0.058824,-0.07926,0.098039,0.056092,0.705882,N/A
1,AAL,17.33,62064479080,0.146614,0.529412,0.067319,0.54902,0.309309,0.960784,0.068627,0.764706,N/A
2,AAP,69.63,15447040391,-0.623538,0.019608,-0.517678,0.019608,-0.452689,0.019608,0.033514,0.45098,N/A
3,AAPL,193.11,5569765535,0.272854,0.745098,0.406835,0.921569,0.162104,0.823529,0.04389,0.568627,N/A
4,ABBV,143.35,7503156068,0.001818,0.215686,-0.03009,0.235294,-0.111127,0.039216,0.043145,0.529412,N/A
5,ABC,194.16,5539644738,0.387965,0.921569,0.19945,0.784314,0.168746,0.843137,0.04835,0.607843,N/A
6,ABMD,381.02,2822889671,0,0.186275,0,0.303922,0,0.264706,0,0.205882,N/A
7,ABT,113.77,9453963456,0.053077,0.333333,0.000956,0.333333,0.018272,0.313725,0.053105,0.647059,N/A
8,ACN,314.88,3415832769,0.127349,0.490196,0.122528,0.666667,0.138102,0.705882,-0.011959,0.137255,N/A
9,ADBE,523.5,2054589154,0.287404,0.784314,0.450362,0.941176,0.35928,0.980392,0.063846,0.72549,N/A


In [41]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        # Convert the strings to floats using float(), but exclude 'N/A' values
        percentile_value = hqm_dataframe.loc[row, f'{time_period} Return Percentile']
        if percentile_value != 'N/A':
            momentum_percentiles.append(float(percentile_value))
    if momentum_percentiles:
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.42,N/A,0.046172,N/A,-0.198364,N/A,-0.07926,N/A,0.056092,N/A,N/A
1,AAL,17.33,N/A,0.146614,N/A,0.067319,N/A,0.309309,N/A,0.068627,N/A,N/A
2,AAP,69.63,N/A,-0.623538,N/A,-0.517678,N/A,-0.452689,N/A,0.033514,N/A,N/A
3,AAPL,193.11,N/A,0.272854,N/A,0.406835,N/A,0.162104,N/A,0.04389,N/A,N/A
4,ABBV,143.35,N/A,0.001818,N/A,-0.03009,N/A,-0.111127,N/A,0.043145,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,XLNX,194.92,N/A,0,N/A,0,N/A,0,N/A,0,N/A,N/A
492,XOM,103.74,N/A,0.198852,N/A,-0.072696,N/A,-0.098,N/A,0.006717,N/A,N/A
493,XRAY,40.88,N/A,0.145058,N/A,0.109847,N/A,0.000705,N/A,0.042293,N/A,N/A
494,XRX,15.49,N/A,0.049897,N/A,-0.038604,N/A,0.153423,N/A,0.110412,N/A,N/A


In [58]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [59]:
portfolio_input()

Enter the value of your portfolio:4525558445588


In [60]:
print(len(hqm_dataframe.index))

51


In [61]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.42,701917735,0.046172,0.294118,-0.198364,0.058824,-0.07926,0.098039,0.056092,0.705882,N/A
1,AAL,17.33,5120394697,0.146614,0.529412,0.067319,0.54902,0.309309,0.960784,0.068627,0.764706,N/A
2,AAP,69.63,1274399542,-0.623538,0.019608,-0.517678,0.019608,-0.452689,0.019608,0.033514,0.45098,N/A
3,AAPL,193.11,459512402,0.272854,0.745098,0.406835,0.921569,0.162104,0.823529,0.04389,0.568627,N/A
4,ABBV,143.35,619019463,0.001818,0.215686,-0.03009,0.235294,-0.111127,0.039216,0.043145,0.529412,N/A
5,ABC,194.16,457027400,0.387965,0.921569,0.19945,0.784314,0.168746,0.843137,0.04835,0.607843,N/A
6,ABMD,381.02,232891816,0,0.186275,0,0.303922,0,0.264706,0,0.205882,N/A
7,ABT,113.77,779963435,0.053077,0.333333,0.000956,0.333333,0.018272,0.313725,0.053105,0.647059,N/A
8,ACN,314.88,281810340,0.127349,0.490196,0.122528,0.666667,0.138102,0.705882,-0.011959,0.137255,N/A
9,ADBE,523.5,169506093,0.287404,0.784314,0.450362,0.941176,0.35928,0.980392,0.063846,0.72549,N/A


In [62]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [63]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [64]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [65]:
writer.save()